# Accessing data from the Materials Project (next-gen)

[API documentation](https://api.materialsproject.org/docs#/Summary/search_summary__get)

In [1]:
import pprint

from mp_api.client import MPRester


/Users/aonwu/.conda/envs/m3gnet/lib/python3.9/site-packages/pymatgen/analysis/phase_diagram.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Getting structures

Let's say we want to find all the structures which contained Lithium and had a band gap higher than 1 eV. We can directly query the MP following the structure of the API.
To query for a particular element, we use the `elements` parameter. To query for a particular band gap value we use the `band_gap` parameter. The criteria passed to `MPRester` is as follows:

For the parameters that can be used in a Materials Project query, see the documentation (https://github.com/materialsproject/mapidoc/tree/master/materials)

For a full description of the MongoDB query operators, please refer to the MongoDB manual (https://www.mongodb.com/docs/manual/reference/operator/query/). 
Being able to use these operators will enhance your querying.

In [2]:
# This defines the query to the materials project
criteria = {'elements': {'$all':['Li']}, 'band_gap':{'$gte':1}}

# This defines the properties that we are interested in
fields= ['material_id', 'formula_pretty', 'structure' ]

# Performing the query

with MPRester('4Ib91crOo7Uwxc0J021oHawyASCKnIVr') as mpr:
    docs = mpr.summary.search(elements=['Li'], band_gap=(1, None),
     fields =fields)

print(len(docs))

/Users/aonwu/.conda/envs/m3gnet/lib/python3.9/site-packages/mp_api/client/client.py:139: builtins.UserWarning: Problem loading MPContribs client: [<twisted.python.failure.Failure OpenSSL.SSL.Error: [('STORE routines', '', 'unregistered scheme'), ('system library', '', ''), ('STORE routines', '', 'unregistered scheme'), ('system library', '', ''), ('STORE routines', '', 'unregistered scheme'), ('system library', '', ''), ('SSL routines', '', 'certificate verify failed')]>]
Retrieving SummaryDoc documents: 100%|██████████| 9340/9340 [00:26<00:00, 351.75it/s]

9340


In [3]:
# data is list of dictionaries
pprint.pprint(docs[0])

MPDataDoc<SummaryDoc>(
formula_pretty='LiSn2P3O10',
material_id=MPID(mp-673134),
structure=Structure Summary
Lattice
    abc : 9.218717 5.079296 10.051037787499608
 angles : 85.36606560175449 90.0 90.0
 volume : 469.0973353956243
      A : 9.218717 0.0 0.0
      B : 0.0 5.079296 0.0
      C : 0.0 0.812016 10.018183
    pbc : True True True
PeriodicSite: Li (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Li (4.6094, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: Sn (4.0760, 0.6861, 6.7484) [0.4421, 0.0274, 0.6736]
PeriodicSite: Sn (5.1428, 5.2052, 3.2698) [0.5579, 0.9726, 0.3264]
PeriodicSite: Sn (8.6853, 5.2052, 3.2698) [0.9421, 0.9726, 0.3264]
PeriodicSite: Sn (0.5334, 0.6861, 6.7484) [0.0579, 0.0274, 0.6736]
PeriodicSite: P (6.9140, 4.2761, 8.3848) [0.7500, 0.7081, 0.8370]
PeriodicSite: P (6.9140, 2.6995, 5.6883) [0.7500, 0.4407, 0.5678]
PeriodicSite: P (2.3047, 3.1918, 4.3299) [0.2500, 0.5593, 0.4322]
PeriodicSite: P (6.9140, 3.0585, 1.2391) [0.7500, 0.5824, 

Using some additional MongoDB operators we could refine our query. 
For example, we could filter out radioactivate elements and trainsition metals in our query using the `'$nin'` operator.

In [4]:
# A list of radioactive elements
radioactive_elements=['Tc', 'Pm', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr']

# A list of transition metal elements excluding Scandium (Sc), Yttrium (Y), Zirconium (Zr) and Niobium (Nb)
transition_metals = ['Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'La', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Ac']

# Merge the lists
not_wanted = radioactive_elements + transition_metals


# Define the properties we are interested in
fields= ['material_id', 'pretty_formula', 'structure' ]

# Query the MP
with MPRester('4Ib91crOo7Uwxc0J021oHawyASCKnIVr') as mpr:
    docs = mpr.summary.search(elements=['Li'],
    exclude_elements=not_wanted,
    band_gap=(1,None),
    fields = fields)
print(len(docs))

/Users/aonwu/.conda/envs/m3gnet/lib/python3.9/site-packages/mp_api/client/client.py:139: builtins.UserWarning: Problem loading MPContribs client: [<twisted.python.failure.Failure OpenSSL.SSL.Error: [('STORE routines', '', 'unregistered scheme'), ('system library', '', ''), ('STORE routines', '', 'unregistered scheme'), ('system library', '', ''), ('STORE routines', '', 'unregistered scheme'), ('system library', '', ''), ('SSL routines', '', 'certificate verify failed')]>]
Retrieving SummaryDoc documents: 100%|██████████| 2134/2134 [00:04<00:00, 481.42it/s]

2134
